### Création d’un trigger sur une table

On veut un trigger qui met à jour automatiquement la colonne updated_at à chaque modification :

- Ajouter la colonne si elle n’existe pas

```sql
ALTER TABLE produits ADD COLUMN updated_at TIMESTAMP DEFAULT NOW();
```

- Créer la fonction du trigger

```sql
CREATE OR REPLACE FUNCTION maj_updated_at()
RETURNS TRIGGER AS $$
BEGIN
    NEW.updated_at := NOW();
    RETURN NEW;
END;
$$ LANGUAGE plpgsql;
```

- Associer la fonction à un trigger

```sql
CREATE TRIGGER trigger_maj_updated_at
BEFORE UPDATE ON produits
FOR EACH ROW EXECUTE FUNCTION maj_updated_at();
```

= Maintenant, chaque fois qu’un produit est modifié, la colonne updated_at est mise à jour.

- exemple:

```sql
UPDATE produits SET prix = 150 WHERE id = 1;
SELECT * FROM produits WHERE id = 1;
```

= Le champ updated_at est mis à jour automatiquement !

### Trigger AVANT une action

Un trigger BEFORE permet de modifier les données avant leur insertion.

- Empêcher un prix négatif

```sql
CREATE OR REPLACE FUNCTION check_prix()
RETURNS TRIGGER AS $$
BEGIN
    IF NEW.prix < 0 THEN
        RAISE EXCEPTION 'Le prix ne peut pas être négatif !';
    END IF;
    RETURN NEW;
END;
$$ LANGUAGE plpgsql;

CREATE TRIGGER trigger_check_prix
BEFORE INSERT OR UPDATE ON produits
FOR EACH ROW EXECUTE FUNCTION check_prix();
```

= PostgreSQL bloque les prix négatifs automatiquement.

### Trigger APRÈS une action

Un trigger AFTER est utile pour auditer ou synchroniser des données après une modification.

- Enregistrer les suppressions dans un journal

```sql
CREATE TABLE produits_deleted (
    id SERIAL PRIMARY KEY,
    produit_id INT,
    deleted_at TIMESTAMP DEFAULT NOW()
);
```

```sql
CREATE OR REPLACE FUNCTION log_suppression_produit()
RETURNS TRIGGER AS $$
BEGIN
    INSERT INTO produits_deleted (produit_id) VALUES (OLD.id);
    RETURN OLD;
END;
$$ LANGUAGE plpgsql;

CREATE TRIGGER trigger_log_delete
AFTER DELETE ON produits
FOR EACH ROW EXECUTE FUNCTION log_suppression_produit();
```

= Toutes les suppressions de produits sont enregistrées dans produits_deleted.

### Triggers pour Synchroniser PostgreSQL et Elasticsearch

L’objectif est de mettre à jour Elasticsearch automatiquement quand une modification est faite dans PostgreSQL.

- Ajouter un produit dans Elasticsearch après son insertion

On va appeler une API avec http_post (via pg_cron ou pg_notify dans un vrai projet).

```sql
CREATE OR REPLACE FUNCTION sync_elasticsearch()
RETURNS TRIGGER AS $$
DECLARE
    json_data TEXT;
BEGIN
    json_data := json_build_object(
        'id', NEW.id,
        'nom', NEW.nom,
        'prix', NEW.prix
    )::TEXT;

    PERFORM pg_notify('elasticsearch_sync', json_data);
    RETURN NEW;
END;
$$ LANGUAGE plpgsql;

CREATE TRIGGER trigger_sync_elasticsearch
AFTER INSERT OR UPDATE ON produits
FOR EACH ROW EXECUTE FUNCTION sync_elasticsearch();
```

PostgreSQL envoie un message à un listener (pg_notify) qui peut déclencher une mise à jour Elasticsearch.

|Action|Fonction / Trigger|
|:---|:---|
|Créer une fonction stockée|```CREATE FUNCTION```|
|Automatiser une mise à jour (ex: updated_at)|```BEFORE UPDATE``` trigger|
|Empêcher des données invalides (ex: prix négatif)|```BEFORE INSERT OR UPDATE``` trigger|
|Enregistrer les suppressions|```AFTER DELETE``` trigger|
|Synchroniser PostgreSQL avec Elasticsearch|```AFTER INSERT OR UPDATE``` trigger + ```pg_notify```|